In [1]:
# crop the faces so that the face size iis bigger (interpupil distance from ) close to real data

In [2]:
# used to process the downloaded synthesized iamge and json files
# to create face images and label files as input to L2CS model training
# cdata/Image and cdata/Label are created and populated
# takes about 100 min

In [3]:
import os
import sys
import numpy as np
from PIL import Image
from pytz import timezone
import arrow

In [4]:
now=arrow.utcnow().to('US/Pacific')
print(now)

2022-06-21T17:55:12.423218-07:00


In [5]:
!pwd

/Users/jyao/JIAN/work/SAI/projects/2022/Synthesize_gaze_data/seventy5k


In [6]:
sys.path.append('/Users/jyao/JIAN/work/SAI/projects/modules')
from utils import SynJSON as SJ

In [7]:
#source data path
sdatapath = '/Users/jyao/JIAN/work/SAI/projects/2022/Synthesize_gaze_data/seventy5k/download/'
# destination data path
ddatapath = '/Users/jyao/JIAN/work/SAI/projects/2022/Synthesize_gaze_data/seventy5k/adata/'
labelpath = os.path.join(ddatapath, 'Label')
imagepath = os.path.join(ddatapath, 'Image')
leftpath = os.path.join(imagepath, 'left')
rightpath = os.path.join(imagepath, 'right')
imagepath = os.path.join(imagepath, 'face')
os.makedirs(labelpath, exist_ok=True)
os.makedirs(imagepath, exist_ok=True)
os.makedirs(leftpath, exist_ok=True)
os.makedirs(rightpath, exist_ok=True)
label_title = 'Face Left Right Origin WhichEye 3DGaze 3DHead 2DGaze 2DHead Rmat Smat GazeOrigin'

In [8]:
%%time
final_size = 224
ratio = 0.90
half_size = int(final_size*0.5*ratio)

for fold in range(15):
    foldstr = f'fold{fold:0>2}'
    label_file = f'p{fold:0>2}.label'
    label_file = os.path.join(labelpath, label_file)
    lf = open(label_file, "w")
#     print(label_file)
    
    sfoldpath = os.path.join(sdatapath, foldstr)
    dfoldpath = os.path.join(imagepath, foldstr)
    os.makedirs(dfoldpath, exist_ok=True)
#     print(dfoldpath)

    jfiles = os.listdir(sfoldpath)
    jfiles = [f for f in jfiles if f.endswith('json')]
    jfiles.sort()
    print(f"fold {fold}, n_jfiles={len(jfiles)}")
    now=arrow.utcnow().to('US/Pacific')
    print(now)
    
    
    
    lf = open(label_file, "w")
#     print(label_title, file=lf)
    
    for i, jfile in enumerate(jfiles):
        if (i+1)%100==0:
            print(f'{i+1}', end=' ')
        imfile = jfile.replace('info.json', 'rgb.png')
#         print(jfile, imfile)
    #     jfile = '3.cam_visible_camera.f_1.info.json'
        jfile = os.path.join(sfoldpath,jfile)
    #     imfile ='3.cam_visible_camera.f_1.rgb.png'
        imfile = os.path.join(sfoldpath, imfile)
        
        #crop image and save image
        sj = SJ(jfile)
        face_center = sj.face_center()
        im = Image.open(imfile)
#         display(im)
        assert im.size[0] == im.size[1]
        center = face_center*im.size[0]
        center = center.astype(int)
        crop_position = np.concatenate([center-half_size, center+half_size])
        m1 = im.crop(crop_position)
        m1 = m1.resize((final_size, final_size))
                
        
        dimfile = os.path.join(dfoldpath, str(i+1)+'.jpeg')
        m1.save(dimfile)
       
        #write label:
        face = '/'.join(dimfile.split('/')[-3:])
        left = 'NA'
        right = 'NA'
        origin = '/'.join(imfile.split('/')[-2:])
        pitch, yaw = sj.pitch_yaw()
#         print(pitchyaw)
        whicheye = 'NA'
        d3gaze = '0.0,0.0,0.0'
        d3head = '0.0,0.0,0.0'
        d2gaze = f'{pitch},{yaw}'
        d2head = '0.0,0.0'
        rmat=smat = '1.0,1.0,1.0'
        gazeorigin = '0.0,0.0,0.0'
        print(' '.join([face, left, right, origin, whicheye, d3gaze,\
                        d3head, d2gaze, d2head, rmat, smat, gazeorigin]), file=lf)

    lf.close()

fold 0, n_jfiles=3000
2022-06-21T17:55:17.331076-07:00
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 fold 1, n_jfiles=3000
2022-06-21T18:00:10.728974-07:00
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 fold 2, n_jfiles=3000
2022-06-21T18:04:53.289704-07:00
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 fold 3, n_jfiles=3000
2022-06-21T18:09:09.301930-07:00
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 fold 4, n_jfiles=3000
2022-06-21T18:14:56.490977-07:00
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 fold 5, n_jfiles=300

In [9]:
!pwd

/Users/jyao/JIAN/work/SAI/projects/2022/Synthesize_gaze_data/seventy5k


In [10]:
sys.exit()

SystemExit: 

/Users/jyao/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
